In [ ]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
cp.fuse(np.float16)
h = 1.

# Get the current free memory on CUDA device
free_memory = cp.cuda.Device().mem_info[0]

def expected_var(v):
    return cp.sum(cp.square(v))

In [ ]:
v= cp.array([-0.00606533, -0.02837768, -0.20481078, -0.05524456,  0.00408442, -0.02378791, -0.11289296, -0.09047946, -0.0828985,   0.01015773]) # Real life
v_0 = cp.array([-0.26251362, -0.22397083, -0.28459696, -0.14160629,  0.11507459,
       -0.01314795,  0.00368215, -0.2233519 , -0.0494188 , -0.09833207])
v_0 = cp.array([ 1.02602951,  1.59187996,  0.23337749, -0.62634188,  0.16052645,
       -0.47135837,  0.21609987,  0.45909724,  0.70707697, -0.76436048])
v_1 = cp.array([-0.09152057,  0.26501426, -0.26361748, -0.09584528,  0.07564258,
       -0.28932995, -0.15172387, -0.17311338, -0.02250007, -0.02662765])
v_2 = cp.array([ 0.02806491,  0.18164134, -0.02569311,  0.08406244,  0.09760685,
       -0.2754576 , -0.18692242,  0.05429335, -0.05959692, -0.16104073])
v_3 = cp.array([ 0.00719622, -0.01367537, -0.04378771,  0.15642576,  0.03295938,
       -0.1364489 , -0.02709714, -0.16822205, -0.15617831, -0.05832736])

v_array = [v_0, v_1, v_2, v_3]
exp_var = cp.sum(cp.square(v_3))

C_ell = cp.array([.25,0.5,0.75,1.])

**Either the cell above or below**

In [ ]:
v_1= cp.array([-0.00606533, -0.02837768, -0.20481078, -0.05524456,  0.00408442, -0.02378791, -0.11289296, -0.09047946, -0.0828985,   0.01015773])
v_0 = cp.array([ 0.05289612, -0.03298811, -0.21346561, -0.03713753, -0.06898345,
       -0.12286244, -0.07009093, -0.13369201, -0.19136073, -0.00077765])
v_array = [v_0, v_1]
exp_var = expected_var(v_1)
print("diff =",expected_var(v_1)-expected_var(v_0))

C_ell = cp.array([0.5,1])

In [ ]:
v_1 = cp.array([-0.04775578, -0.02158142, -0.22861181,  0.00999135 , 0.05261623, -0.03810132, -0.08892537, -0.09858141, -0.06851002,  0.09957945])
v_2= cp.array([-0.00606533, -0.02837768, -0.20481078, -0.05524456,  0.00408442, -0.02378791, -0.11289296, -0.09047946, -0.0828985,   0.01015773]) #v_1 => high fidelity
v_0= cp.array([-0.2506599 ,  0.05844203, -0.30284406,  0.08003633,  0.13573025, 0.04745837,  0.05874857,  0.07815323,  0.20496045,  0.16683998])

v_array = [v_0, v_1, v_2]
exp_var = expected_var(v_array[-1])
C_ell = cp.array([.50,0.70,1])

In [ ]:
# Generating vectors to have a variance close to the real one v
eps = 10e-2
v_generated = v_1 + cp.random.normal(0,eps,len(v))
print("|V - V_generated| =", cp.linalg.norm(expected_var(v_generated) - expected_var(v)))
print("exp_var v_generated =" , expected_var(v_generated))
print("diff =" , expected_var(v_1)-expected_var(v_generated))
v_generated

In [ ]:
# Variable & functions def: 
def f_vec(X,v):
    return v.T@X

def f_vec_squared(X,v):
    return (v.T@X)**2

d = 10 # lenght of random vectors

In [ ]:
# Function that computes the estimator of the variance in ML & LML :
def estim_var_MLMC_tqdm(N, d, M, batch_size):
    var_results = []
    for i in tqdm(range(0, N, batch_size)):
        actual_batch_size = min(batch_size, N - i)
        # Variance at level 0
        X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[0]))
        # Y_ell_M_ell = f_vec(X_M_ell,v_0)
        Var_MLMC_batch = cp.var(f_vec(X_M_ell,v_0), axis=1, ddof=1)
        X_M_ell = None
        # Loop on M to have correction V^(ell)_{M_ell}(Y_ell) - V^(ell)_{M_ell}(Y_{ell-1})
        for ell in range(1,len(M)):
            X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[ell]))
            # Y_ell_M_ell = f_vec(X_M_ell,v_array[ell])
            # Y_ell_minus_1_M_ell = f_vec(X_M_ell,v_array[ell-1])
            Var_MLMC_batch += cp.var(f_vec(X_M_ell,v_array[ell]), axis=1, ddof=1) - cp.var(f_vec(X_M_ell,v_array[ell-1]), axis=1, ddof=1)

        var_results.extend(Var_MLMC_batch)

        # Free memory
        X_M_ell = None
        cp.get_default_memory_pool().free_all_blocks()

    return cp.array(var_results) # return var_results (len = N)

def estim_var_MLMC_log_tqdm(N, d, M, batch_size):
    var_results = []
    for i in tqdm(range(0, N, batch_size)):
        actual_batch_size = min(batch_size, N - i)
        # Variance at level 0
        X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[0]))
        # Y_ell_M_ell = f_vec(X_M_ell,v_0)
        log_Var_log_MLMC = cp.log(cp.var(f_vec(X_M_ell,v_0), axis=1, ddof=1))
        X_M_ell = None
    
        # Loop on M to have correction V^(ell)_{M_ell}(Y_ell) - V^(ell)_{M_ell}(Y_{ell-1})
        for ell in range(1,len(M)):
            X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[ell]))
            # Y_ell_M_ell = f_vec(X_M_ell,v_array[ell])
            # Y_ell_minus_1_M_ell = f_vec(X_M_ell,v_array[ell-1])
            log_Var_log_MLMC += np.log(cp.var(f_vec(X_M_ell,v_array[ell]), axis=1, ddof=1)) - cp.log(cp.var(f_vec(X_M_ell,v_array[ell-1]), axis=1, ddof=1))

        Var_MLMC_log_batch = cp.exp(log_Var_log_MLMC)
    
        var_results.extend(Var_MLMC_log_batch) 

        # Free memory if necessary
        X_M_ell = None
        cp.get_default_memory_pool().free_all_blocks()

    return cp.array(var_results) # return var_results (len = N)

In [ ]:
# Function that computes the estimator of the variance in ML & LML :
def estim_var_MLMC(N, d, M, batch_size):
    var_results = []
    for i in range(0, N, batch_size):
        actual_batch_size = min(batch_size, N - i)
        # Variance at level 0
        X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[0]))
        # Y_ell_M_ell = f_vec(X_M_ell,v_0)
        Var_MLMC_batch = cp.var(f_vec(X_M_ell,v_0), axis=1, ddof=1)
        X_M_ell = None
        # Loop on M to have correction V^(ell)_{M_ell}(Y_ell) - V^(ell)_{M_ell}(Y_{ell-1})
        for ell in range(1,len(M)):
            X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[ell]))
            # Y_ell_M_ell = f_vec(X_M_ell,v_array[ell])
            # Y_ell_minus_1_M_ell = f_vec(X_M_ell,v_array[ell-1])
            Var_MLMC_batch += cp.var(f_vec(X_M_ell,v_array[ell]), axis=1, ddof=1) - cp.var(f_vec(X_M_ell,v_array[ell-1]), axis=1, ddof=1)

        var_results.extend(Var_MLMC_batch)

        # Free memory
        X_M_ell = None
        cp.get_default_memory_pool().free_all_blocks()

    return cp.array(var_results) # return var_results (len = N)

def estim_var_MLMC_log(N, d, M, batch_size):
    var_results = []
    for i in range(0, N, batch_size):
        actual_batch_size = min(batch_size, N - i)
        # Variance at level 0
        X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[0]))
        # Y_ell_M_ell = f_vec(X_M_ell,v_0)
        log_Var_log_MLMC = cp.log(cp.var(f_vec(X_M_ell,v_0), axis=1, ddof=1))
        X_M_ell = None
    
        # Loop on M to have correction V^(ell)_{M_ell}(Y_ell) - V^(ell)_{M_ell}(Y_{ell-1})
        for ell in range(1,len(M)):
            X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[ell]))
            # Y_ell_M_ell = f_vec(X_M_ell,v_array[ell])
            # Y_ell_minus_1_M_ell = f_vec(X_M_ell,v_array[ell-1])
            log_Var_log_MLMC += np.log(cp.var(f_vec(X_M_ell,v_array[ell]), axis=1, ddof=1)) - cp.log(cp.var(f_vec(X_M_ell,v_array[ell-1]), axis=1, ddof=1))

        Var_MLMC_log_batch = cp.exp(log_Var_log_MLMC)
    
        var_results.extend(Var_MLMC_log_batch) 

        # Free memory if necessary
        X_M_ell = None
        cp.get_default_memory_pool().free_all_blocks()

    return cp.array(var_results) # return var_results (len = N)

In [ ]:
# Since we can't compute all at once due to M_0 and M_1 increasing, we need to separate the caluclations by computing batch_size first
def get_max_batch_size(d, M, free_memory, memory_utilization=0.7):
    """
    Calculate the maximum batch size directly based on the memory usage per data sample and the available memory.
    """
    # Calculate the memory usage per sample
    bytes_per_number = cp.dtype('float16').itemsize
    omega = (d+1) * bytes_per_number * (np.sum(M))  # Total memory used per sample

    # Calculate the maximum amount of memory available for batches
    max_memory_per_batch = free_memory * memory_utilization

    # Calculate the maximum batch size that can fit within the memory limit
    batch_size = int(max_memory_per_batch / omega)  # Use int to ensure we get a whole number

    return batch_size


In [ ]:
# N = 1000000
# M = [18]
# X_M_ell = cp.random.standard_normal((N,d,M[0]))
# Y_ell_M_ell_squared = f_vec_squared(X_M_ell,v)
# Y_ell_M_ell = f_vec(X_M_ell,v)

# print(cp.mean(Y_ell_M_ell))

# print("Var en calculant avec cp.var (ddof = 1) =", cp.var(Y_ell_M_ell, axis=1, ddof=1))
# print("Var en calculant avec cp.var + correction =", M[0]/(M[0]-1)*cp.var(Y_ell_M_ell, axis=1))
# print("var en calculant à la main la somme =", cp.sum(Y_ell_M_ell_squared, axis=1)/(M[0]-1))

In [ ]:
def M_ell_tab(eta, V_ell, C_ell,m=0):
    """
    eta : budget
    V_ell : tableau contenant [V_0,V_1,...,V_L]
    C_ell : tableau contenant [C_0,C_1,...,C_L]

    return : la formule M_ell au dessus qui nous donne le nombre d'élement optimal à simuler
    """
    eta_tilde = eta # - m*cp.sum(...)
    S_L = (cp.sqrt(cp.multiply(V_ell,C_ell))).sum()

    return m + 1 + cp.floor(
        (eta_tilde/S_L)*cp.sqrt(V_ell*1/C_ell)
    )

## Nombres de cas patho

In [ ]:
# Run cette cellule si t'as changé les C_ell
N_pilote = 10000
X = cp.random.standard_normal((d,N_pilote))
V = [cp.var(f_vec_squared(X,v_0))]
for i in range(1,len(v_array)):
    V.extend([cp.var(f_vec_squared(X,v_array[i]) - f_vec_squared(X,v_array[i-1]))])

V_ell = cp.array(V)

In [ ]:
budget_eta_int = 1000
N = 100000
d = 10
M = list(map(int,M_ell_tab(budget_eta_int,V_ell,C_ell)))
# print(M)
batch_size = get_max_batch_size(d, M, free_memory)
print(batch_size)
print("M_ell :", M)

In [ ]:
Tab_var_MLMC = estim_var_MLMC_tqdm(N, d, M, batch_size)
cp.mean(Tab_var_MLMC)
#Tab_var_MLMC = np.array([x.item() for x in Tab_var_MLMC])

In [ ]:
Tab_var_log_MLMC = estim_var_MLMC_log_tqdm(N, d, M, batch_size)
#Tab_var_log_MLMC = np.array([x.item() for x in Tab_var_log_MLMC])

In [ ]:
# # print(cp.sum(Tab_var_MLMC_log < 0))
# bias_MLMC_tab = []
# tab_neg_val = []
# bias_log_MLMC_tab = []
# for _ in range(100):
#     Tab_var_MLMC = estim_var_MLMC(N, d, M, batch_size)
#     bias_MLMC_tab.append(cp.asnumpy(cp.abs(cp.mean(Tab_var_MLMC)-exp_var)))
#     tab_neg_val.append(cp.asnumpy(cp.sum(Tab_var_MLMC < 0)))
#     Tab_var_MLMC = None

#     Tab_var_log_MLMC = estim_var_MLMC_log(N, d, M, batch_size)
#     bias_log_MLMC_tab.append(cp.asnumpy(np.abs(np.mean(Tab_var_log_MLMC)-exp_var)))
#     Tab_var_log_MLMC = None

# print(f"{np.mean(bias_MLMC_tab):.2e}")
# print(format(np.mean(bias_log_MLMC_tab), ".2e"))
# print(cp.mean(cp.array(tab_neg_val)))

In [ ]:
print("$V^{ML}$ bias =",f"{cp.asnumpy(cp.abs(cp.mean(Tab_var_MLMC)-exp_var)):.2e}")
print(f"{cp.mean(cp.asnumpy(cp.sum(Tab_var_MLMC < 0)))/N*100}")
print("$V^{ML}$ bias =",f"{cp.asnumpy(np.abs(np.mean(Tab_var_log_MLMC)-exp_var)):.2e}")

In [ ]:
# np.save(f'save_temp_data_hist\Tab_var_MLMC_eta_{budget_eta_int}', Tab_var_MLMC)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5))

# Histogramme et ligne de variance attendue
ax1.hist(cp.asnumpy(Tab_var_MLMC), bins=30)
ax1.set_xlim([-0.075, 0.25])
ax1.set_ylim([0, 1330])
ax1.axvline(x=cp.asnumpy(exp_var), color='r', linestyle='--', linewidth=1.25, label='Expected Variance')
ax1.axvline(x=np.mean(cp.asnumpy(Tab_var_MLMC)), color='lime', linestyle='-', linewidth=1, label='Mean Value of $\hat{V}^{ML}_L$')

# Ajouter une zone hachurée
ax1.axvspan(xmin=-0.075, xmax=0, ymin=0, ymax=1, facecolor='gray', alpha=0.2, hatch='//')

ax2.hist(cp.asnumpy(Tab_var_log_MLMC), bins=30)
ax2.set_xlim([-0.075, 0.25])
ax2.set_ylim([0, 1330])
ax2.axvline(x=cp.asnumpy(exp_var), color='r', linestyle='--', linewidth=1.25, label='Expected Variance')
ax2.axvline(x=np.mean(cp.asnumpy(Tab_var_log_MLMC)), color='lime', linestyle='-', linewidth=1, label='Mean Value of $\hat{V}^{LML}_L$')

# Ajouter une zone hachurée
ax2.axvspan(xmin=-0.075, xmax=0, ymin=0, ymax=1, facecolor='gray', alpha=0.2, hatch='//')


# Ajout de légendes
ax1.legend()
ax2.legend()

# Ajout des x et y labels
ax1.set_xlabel('$\hat{V}^{ML}_L$')
ax1.set_ylabel('Count')

ax2.set_xlabel('$\hat{V}^{LML}_L$')
# ax2.set_ylabel('Count')

# Ajout des titles
ax1.set_title('Distribution of $\hat{V}^{ML}_L$') 
ax2.set_title('Distribution of $\hat{V}^{LML}_L$')

fig.suptitle(r'Comparison of the distribution for estimates $\hat{V}^{ML}_L$ and $\hat{V}^{LML}_L$ for a low budget $B =$' + f'{budget_eta_int}')
fig.tight_layout(rect=[0, 0, 1, 0.96])

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5))

# Histogramme et ligne de variance attendue
ax1.hist(cp.asnumpy(Tab_var_MLMC), bins=30)
ax1.set_xlim([-0.2, .325])
ax1.set_ylim([0, 1920])
ax1.axvline(x=cp.asnumpy(exp_var), color='r', linestyle='--', linewidth=1.25, label='Expected Variance')
ax1.axvline(x=np.mean(cp.asnumpy(Tab_var_MLMC)), color='lime', linestyle='-', linewidth=1, label='Mean Value of $\hat{V}^{ML}_L$')

# Ajouter une zone hachurée
ax1.axvspan(xmin=-0.2, xmax=0, ymin=0, ymax=1, facecolor='gray', alpha=0.2, hatch='//')

ax2.hist(cp.asnumpy(Tab_var_log_MLMC), bins=30)
ax2.set_xlim([-0.2, .325])
ax2.set_ylim([0, 1920])
ax2.axvline(x=cp.asnumpy(exp_var), color='r', linestyle='--', linewidth=1.25, label='Expected Variance')
ax2.axvline(x=np.mean(cp.asnumpy(Tab_var_log_MLMC)), color='lime', linestyle='-', linewidth=1, label='Mean Value of $\hat{V}^{LML}_L$')

# Ajouter une zone hachurée
ax2.axvspan(xmin=-0.2, xmax=0, ymin=0, ymax=1, facecolor='gray', alpha=0.2, hatch='//')


# Ajout de légendes
ax1.legend()
ax2.legend()

# Ajout des x et y labels
ax1.set_xlabel('$\hat{V}^{ML}_L$')
ax1.set_ylabel('Count')

ax2.set_xlabel('$\hat{V}^{LML}_L$')
# ax2.set_ylabel('Count')

# Ajout des titles
ax1.set_title('Distribution of $\hat{V}^{ML}_L$') 
ax2.set_title('Distribution of $\hat{V}^{LML}_L$')

fig.suptitle(r'Comparison of the distribution for estimates $\hat{V}^{ML}_L$ and $\hat{V}^{LML}_L$ for a low budget $B =$' + f'{budget_eta_int}')
fig.tight_layout(rect=[0, 0, 1, 0.96])

plt.show()

In [ ]:
Tab_var_MLMC_log = estim_var_MLMC_log(N, d, M, batch_size)

In [ ]:
Tab_var_MLMC_log = [x.item() for x in Tab_var_MLMC_log]

In [ ]:
plt.hist(Tab_var_MLMC_log)

# MSE en fonction de n
budget fixé à $10^3$

In [ ]:
budget_eta_int = 1e3
samples_N = list(map(int,np.logspace(2,5,5)))

In [ ]:
bias_tab = []
var_MLMC_tab = []
bias_log_tab = []
var_MLMC_log_tab = []


for N in tqdm(samples_N):
    # Pour chaque budget eta, calcul de nos M_0 et M_1
    M = list(map(int,M_ell_tab(budget_eta_int,V_ell,C_ell)))

    # Compute the batch size
    batch_size = get_max_batch_size(d, M, free_memory)
    if batch_size == 0:
        break  # Exit if batch size is too small
    
    # Calcul de nos tableaux de nos estimateurs de taille N
    Tab_var_MLMC = estim_var_MLMC(N, d, M, batch_size)
    var_MLMC_tab.append(cp.var(Tab_var_MLMC))
    bias_tab.append(cp.square(cp.mean(Tab_var_MLMC) - expected_var(v)))

    Tab_var_MLMC = None
    cp.get_default_memory_pool().free_all_blocks()

    Tab_var_MLMC_log = estim_var_MLMC_log(N, d, M, batch_size)
    var_MLMC_log_tab.append(cp.var(Tab_var_MLMC_log))
    bias_log_tab.append(cp.square(cp.mean(Tab_var_MLMC_log) - expected_var(v)))

    Tab_var_MLMC_log = None
    cp.get_default_memory_pool().free_all_blocks()

var_MLMC_tab = [x.item() for x in var_MLMC_tab]
bias_tab = [x.item() for x in bias_tab]

var_MLMC_log_tab = [x.item() for x in var_MLMC_log_tab]
bias_log_tab = [x.item() for x in bias_log_tab]

In [ ]:
# Plot log-log
plt.figure(figsize=(10, 6))

# Plot des données
plt.loglog(samples_N, bias_tab, marker='o', label='MSE MLMC')
#plt.loglog(samples_N, var_MLMC_tab, marker='s', label='Var MLMC')
plt.loglog(samples_N, bias_log_tab, marker='^', label='MSE Log-MLMC', c='g')
#plt.loglog(samples_N, var_MLMC_log_tab, marker='x', label='Var Log-MLMC')


# Ajouter titre et légendes
plt.title(f'MSE et variance en fonction des N pour budget={budget_eta_int}')
plt.xlabel('N')
plt.ylabel('MSE/Variance')
plt.legend()

# Afficher le plot
plt.grid(True)
plt.show()

In [ ]:
teta_hat = cp.random.exponential(30,10)
cp.mean(cp.log(teta_hat))

In [ ]:
cp.log(cp.mean(teta_hat))

In [ ]:
cp.random.exponential(2,10)

# Implémentation du Bootstrap dans le calcul à chaque niveau

$$
\begin{equation}
    \mathbb V = \log \left( x \right)
\end{equation}
$$

In [ ]:
def Bootstrap_var(Y, num_samples):
    """
    This function computes the variance of Y using the bootstrap method.
    It creates num_samples of variances by taking randomly M_ell (= len(Y)) values in Y, and then returning its variance.
    Then it return the expectation of this variances
    
    Parameters:
    - Y: numpy array, values that we want the variance
    - num_samples: int, the number of bootstrap samples to generate
    
    Returns:
    - int, the expectation of bootstrap variances 
    """
    N, M_ell = cp.shape(Y)
    bootstrap_variances = cp.empty((N,num_samples))
    for i in range(num_samples):
        sample_indices = np.random.randint(0, M_ell, M_ell)
        bootstrap_Y = Y[:, sample_indices]
        bootstrap_variances[:,i] = cp.var(bootstrap_Y, axis=1, ddof=1) # corrected empirical variance

    return cp.mean(bootstrap_variances, axis=1)


In [ ]:
def vectorized_bootstrap_var(Y, num_samples):
    """
    This function computes the variance of Y using the bootstrap method, vectorized to avoid explicit loops.
    It creates num_samples of variances by taking randomly M_ell (= len(Y)) values in Y, and then returns its variance.
    Then it returns the expectation of these variances.
    
    Parameters:
    - Y: numpy array, values for which we want the variance
    - num_samples: int, the number of bootstrap samples to generate
    
    Returns:
    - numpy array, the expectation of bootstrap variances for each feature
    """
    N, M_ell = Y.shape
    # Generate random indices for all bootstrap samples at once
    sample_indices = np.random.randint(0, M_ell, (M_ell, num_samples))
    # Index Y to create the bootstrap samples: (N, M_ell, num_samples)
    bootstrap_Y = Y[:, sample_indices]
    # Compute variances across the middle dimension, for each sample
    bootstrap_variances = np.var(bootstrap_Y, axis=1, ddof=1)
    # Compute the mean of the variances across samples
    mean_variances = np.mean(bootstrap_variances, axis=1)
    return mean_variances

# Uncomment the following line to test the function with example data
# vectorized_bootstrap_var(np.random.rand(10, 100), 1000)  # Example call with random data



In [ ]:
# Function that computes the estimator of the variance in ML & LML :
def Estim_var_MLMC_bootstrap(N, d, M, batch_size, bootstrap_sample_size=1000):
    var_results = []
    for i in range(0, N, batch_size):
        actual_batch_size = min(batch_size, N - i)
        # Variance at level 0
        X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[0]))
        Y_ell_M_ell = f_vec(X_M_ell,v_0)
        Var_MLMC_batch = Bootstrap_var(Y_ell_M_ell,bootstrap_sample_size)
    
        # Loop on M to have correction V^(ell)_{M_ell}(Y_ell) - V^(ell)_{M_ell}(Y_{ell-1})
        for ell in range(1,len(M)):
            X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[ell]))
            Y_ell_M_ell = f_vec(X_M_ell,v_array[ell])
            V_ell_M_ell_bootstrap = Bootstrap_var(Y_ell_M_ell,bootstrap_sample_size)

            Y_ell_minus_1_M_ell = f_vec(X_M_ell,v_array[ell-1])
            V_ell_minus_1_M_ell_bootstrap = Bootstrap_var(Y_ell_minus_1_M_ell,bootstrap_sample_size)

            Var_MLMC_batch += V_ell_M_ell_bootstrap - V_ell_minus_1_M_ell_bootstrap

        var_results.extend(Var_MLMC_batch)

        # Free memory
        X_M_ell = None
        cp.get_default_memory_pool().free_all_blocks()

    return cp.array(var_results)

def Estim_var_MLMC_log(N, d, M, batch_size, bootstrap_sample_size):
    var_results = []
    for i in range(0, N, batch_size):
        actual_batch_size = min(batch_size, N - i)
        # Variance at level 0
        X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[0]))
        Y_ell_M_ell = f_vec(X_M_ell,v_0)
        log_Var_log_MLMC = cp.log(cp.var(Y_ell_M_ell, axis=1, ddof=1))
    
        # Loop on M to have correction V^(ell)_{M_ell}(Y_ell) - V^(ell)_{M_ell}(Y_{ell-1})
        for ell in range(1,len(M)):
            X_M_ell = cp.random.standard_normal((actual_batch_size,d,M[ell]))
            Y_ell_M_ell = f_vec(X_M_ell,v_array[ell])
            Y_ell_minus_1_M_ell = f_vec(X_M_ell,v_array[ell-1])
            log_Var_log_MLMC += np.log(cp.var(Y_ell_M_ell, axis=1, ddof=1)) - cp.log(cp.var(Y_ell_minus_1_M_ell, axis=1, ddof=1))

        Var_MLMC_log_batch = cp.exp(log_Var_log_MLMC)
    
        var_results.extend(Var_MLMC_log_batch)

        # Free memory if necessary
        X_M_ell = None
        cp.get_default_memory_pool().free_all_blocks()

    return cp.array(var_results)

In [ ]:
N = 5000
budget_eta_int_bootstrap = 1e3
M = list(map(int,M_ell_tab(budget_eta_int_bootstrap,V_ell,C_ell)))

batch_size = get_max_batch_size(d, M, free_memory)
if batch_size == 0:
    raise ValueError


Tab_var_MLMC = estim_var_MLMC(N, d, M, batch_size)

Tab_var_MLMC_bootstrap = Estim_var_MLMC_bootstrap(N, d, M, batch_size, bootstrap_sample_size=10000)

In [ ]:
Tab_var_MLMC.mean()

In [ ]:
Tab_var_MLMC_bootstrap.mean()

In [ ]:
expected_var(v_3)